In [2]:
import time
import ipywidgets as widgets
from jupyter_ui_poll import ui_events

event_info = {
    'type': '',
    'description': '',
    'time': -1
}


def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

In [3]:
import requests
from bs4 import BeautifulSoup
import json
import time

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


In [5]:
from IPython.display import display, Image, clear_output, HTML
import time
import random
import ipywidgets as widgets



data_consent_info = """DATA CONSENT INFORMATION:

Please read:

we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload."""

print(data_consent_info)
result = input("> ") 

if result == "yes": 
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
    
else: 
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))



id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifer would be CBTC
"""

print(id_instructions)
user_id = input("> ")

print("User entered id:", user_id)


results_dict = {
    'filename': [],
    'nL': [],
    'nR': [],
    'ratio': [],
    'correct': []
}


picture_paths = ["Picture1.png", "Picture2.png", "Picture3.png", "Picture4.png", 
                 "Picture5.png", "Picture6.png", "Picture7.png", "Picture8.png"]

correct_answers = {
    "Picture1.png": "Right",
    "Picture2.png": "Right",
    "Picture3.png": "Left",
    "Picture4.png": "Left",
    "Picture5.png": "Right",
    "Picture6.png": "Left",
    "Picture7.png": "Right",
    "Picture8.png": "Right",
}



def display_dots_test():
    #print("What is your name?")
    #name = input()
    print("What is your age?")
    age = input()
    
    start_time = time.time()
    score = 0
    
    left_button = widgets.Button(description="Left")
    right_button = widgets.Button(description="Right")
    left_button.on_click(register_btn_event)
    right_button.on_click(register_btn_event)

    # Display buttons
    button_container = widgets.HBox([left_button, right_button])

    
    for picture_path in picture_paths: 
        picture = Image(filename=picture_path)
        
        display(picture)
        time.sleep(0.75)  
        clear_output(wait=False)

        display(button_container)
        result = wait_for_event(timeout=3)
        print(result['description'])
        correct_side = correct_answers[picture_path]
        if result['description'] == correct_side:
            score += 1
            print("Well done")
        elif result['description'] == "":
            print("too slow")
        else:
            print("Wrong")
        time.sleep(1.5)


    end_time = time.time()  
    duration = end_time - start_time

    print(f"\nTest completed in {duration:.2f} seconds.")
    print(f"Your score is: {score}/{8}")

    
    results.dict['filename'].append(filename)
    results.dict['nR'].append(nR)
    results.dict['nL'].append(nL)
    results.dict['nR'].append(ratio)
    results.dict['nL'].append(correct)


    data_dict = {
        'user_id': user_id,
        'age': age,
        'score': score,
        'duration': duration
    }
    print(data_dict)

    form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdIpedRAv-rOC6lWVcWzL-bOkGZ9LSWigVYfbM5Q2iTJN9nQQ/viewform?usp=sf_link"  
    send_to_google_form(data_dict, form_url)

    
display_dots_test()



Left
Wrong

Test completed in 30.61 seconds.
Your score is: 6/8


NameError: name 'results' is not defined